# How to make interactive charts.

https://towardsdatascience.com/interactive-graphs-in-python-830b1e6c197f


In [59]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
from ipywidgets import interact, IntSlider, FloatSlider

In [2]:
@interact
def sinewave(c = 1):
    p = np.linspace(0, 2*math.pi, 100)
    y = np.sin(p*c)

    plt.plot(p, y)
    plt.ylabel('sin(x)')
    plt.xlabel('x')
    plt.title('Sinwave function')
    return plt.figure()

interactive(children=(IntSlider(value=1, description='c', max=3, min=-1), Output()), _dom_classes=('widget-int…

# Uniswap invariant - 2 tokens

To automatically market make, we must "move" the markt ourselves after each trade. Clearly, if someony buys token Y, then we have less of it, and so its price must go up. Same goes for token X

To achieve this, we can use an invariant (in mathematical lingo, this means that this piece of truth always holds, no matter what)

Such an invariant may be chosen to be

$$xy = k \ \ \ \ x,y,k \in \mathbb{R^+}$$


i.e. after we multiply the quantity of token X by the quantity of token Y, we will get some value. And this value we must hold constant after each trade.

Example

- deposit 1 x and 2 y into the contract. This sets the initial exchange rate. i.e. for 1 x get 2 y, or equivalently, for 2 y get 1 x
- in this case, we have that $k = 1 * 2 = 2$
- someone wants X for 1 y. This is not a deposit, this is an exchange. Total quantity of Y is now 3, but we need to keep $k=2$. We can do that by giving them back $x = k / y$, $x = 2 / 3$, i.e. $0.\dot{6}$ x

In [3]:
class UniswapAmm:
    # x and y are initial quantities of x and y that set the initial exchange rate
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.k = x * y

    # you are a LP - liquidity provider
    # the more liquidity - the lower the price slippage
    def deposit(self, dx = 0, dy = 0):
        if dx < 0: raise Exception("must be positive")
        if dy < 0: raise Exception("must be positive")
        # I don't know if exchange rate can change, probably
        # since one needs to rebalance
        # but for simplicity's sake, let's disallow that for now
        if dx / dy != self.x / self.y: raise Exception("exchange rate is incorrect")

        self.x += dx
        self.y += dy

    # only minus args. You are either selling X or Y
    def trade(self, x = 0, y = 0):
        minus_x = -x
        minus_y = -y

        if minus_x > self.x: raise Exception("impossible")
        if minus_y > self.y: raise Exception("impossible")
        if minus_x > 0:
            if minus_y > 0: raise Exception("impossible")

            new_x = self.x + minus_x
            y_to_send = self.k / new_x
            new_y = self.y - y_to_send

            self.x = new_x
            self.y = new_y


Some questions we may ask ourselves

- what is the slippage, for varying amounts of tokens bought?
- how do we improve the slippage?
- can this framework be as useful for stablecoins?


# Curve StableSwap - Interpolation between the constant product and constant sum invariants

Constant product invariant is the following

$$\prod_i x_i = \left( \frac{D}{n} \right)^n$$

$$x_i = \left( \frac{D}{n} \right)^n / \prod_{j, j \neq i} x_j$$

In [68]:
# qty of coin a, qty of coin b are the arguments
def product_two_coins(coin_a = 3, coin_b = 3):
    constant = ((coin_a + coin_b) / 2) ** 2
    coin1 = np.linspace(0, 10, 100)
    y = lambda s_xj: [constant / xj for xj in s_xj]
    coin2 = y(coin1)

    plt.plot(coin1, coin2)
    plt.ylabel('Coin B ($)')
    plt.xlabel('Coin A ($)')
    plt.ylim([0, 10])
    plt.xlim([0, 10])
    plt.title(r'$\prod_{i \in \{1, 2 \}} x_i = (D / 2)^2$')
    plt.figure()

In [69]:
product_two_coins = interact(
    product_two_coins,
    coin_a=IntSlider(min=1, max=10, step=1, value=3),
    coin_b=IntSlider(min=1, max=10, step=1, value=3)
)

interactive(children=(IntSlider(value=3, description='coin_a', max=10, min=1), IntSlider(value=3, description=…

Constant sum invariant is the following

$$\sum_i x_i = D$$

In [64]:
def sum_two_coins(coin_a = 5, coin_b = 5):
    constant = coin_a + coin_b
    coin1 = np.linspace(0, 10, 100)
    y = lambda s_xj: [constant - xj for xj in s_xj]
    coin2 = y(coin1)

    plt.plot(coin1, coin2)
    plt.ylabel('Coin B ($)')
    plt.xlabel('Coin A ($)')
    plt.ylim([0, 10])
    plt.xlim([0, 10])
    plt.title(r'$\sum_{i \in \{ 1,2\}} x_i = D$')
    plt.figure()

In [65]:
sum_two_coins = interact(
    sum_two_coins,
    coin_a=IntSlider(min=1, max=10, step=1, value=2),
    coin_b=IntSlider(min=1, max=10, step=1, value=2)
)

interactive(children=(IntSlider(value=2, description='coin_a', max=10, min=1), IntSlider(value=2, description=…

Now constant-sum is perfect for us, it is infinite leverage and so there is zero slippage, however, this is not feasible in the case of AMM, because we can't run out of inventory. As the stock of some coin approaches zero it must become prohibitively expensive to acquite it. To achieve this, we need our constant product involved here too. Constant product, on the other hand, is a framework of "perfect slippage", i.e. there is zero leverage. This means, that we need something in between, an interpolation of the two

$$\chi D^{n - 1} \sum_i x_i + \prod_i x_i = \chi D^n + \left( \frac{D}{n} \right)^n \tag{1}$$

where $\chi$ is leverage

notice that

$$\chi D^{n-1} \sum_i x_i = \chi D^{n-1} \cdot D = \chi D^n$$

and that

$$\prod_i x_i = \left( \frac{D}{n} \right)^n$$

as before, and so equation $(1)$ holds, LHS = RHS

Note also, that if leverage is zero, $\tag{1}$ becomes a constant-product, the zero leverage, the Uniswap like AMM. However, when $\chi$ (our leverage) is 1, we have a levered constant sum with a plain, nothing changed, constant product invariant.

In fact,

as soon as we set $\chi = 1$ and subsequently increase it, we have that

$$\chi D^n \geq \left( \frac{D}{n} \right)^n, \forall \chi \in \mathbb{R}^+ \ \text{and} \ \chi  \geq 1$$

and this means that constant product invariant becomes less and less powerful very quickly

let's see what this would look like

$$\chi D^{n - 1} \sum_i x_i + \prod_i x_i = \chi D^n + \left( \frac{D}{n} \right)^n \tag{1}$$
$$\chi D^{n-1} x_i + \chi D^{n-1} \sum_{j, j \neq i} x_j + \prod_i x_i = \chi D^n + \left( \frac{D}{n} \right)^n$$
$$\chi D^{n-1} x_i + \prod_i x_i = \chi D^n + \left( \frac{D}{n} \right)^n - \chi D^{n-1} \sum_{j, j \neq i} x_j$$
$$ x_i (\chi D^{n-1} + \prod_{j, j \neq i} x_j) = \chi D^n + \left( \frac{D}{n} \right)^n - \chi D^{n-1} \sum_{j, j \neq i} x_j$$
$$ x_i \frac{\left(\chi D^{n-1} + \prod_{j, j \neq i} x_j\right)}{D^n} = \chi + \frac{1}{n^n} - \frac{\chi}{D} \sum_{j, j \neq i} x_j$$
$$ x_i \frac{\left(\chi D^{n-1} + \prod_{j, j \neq i} x_j\right)}{D^n} = \chi \left( 1 - \frac{1}{D} \sum_{j, j \neq i} x_j \right) + \frac{1}{n^n}$$
$$ x_i = \left(\chi \left( 1 - \frac{1}{D} \sum_{j, j \neq i} x_j \right) + \frac{1}{n^n}\right) \div \left( \frac{\left(\chi D^{n-1} + \prod_{j, j \neq i} x_j\right)}{D^n} \right)$$

In [61]:
def mod_two_coins(coin_a = 5, coin_b = 5, leverage = 0):
    constant = coin_a + coin_b
    coin1 = np.linspace(0, 10, 100)
    y = lambda s_xj: [(leverage * (1 - xj / constant) + 1/4) / ((leverage * constant + xj) / (constant ** 2)) for xj in s_xj]
    coin2 = y(coin1)

    plt.plot(coin1, coin2)
    plt.ylabel('Coin B')
    plt.xlabel('Coin A')
    plt.ylim([0, 10])
    plt.xlim([0, 10])
    plt.title(r'$\chi D^{n - 1} \sum_i x_i + \prod_i x_i = \chi D^n + \left( \frac{D}{n} \right)^n$')
    plt.figure()

In [63]:
mod_two_coins = interact(
    mod_two_coins,
    coin_a=IntSlider(min=1, step=1, value=2, max=10),
    coin_b=IntSlider(min=1, step=1, value=2, max=10),
    leverage=FloatSlider(min=0, step=0.01, value=0, max=5)  # after some value, constant product vanishes
)

interactive(children=(IntSlider(value=2, description='coin_a', max=10, min=1), IntSlider(value=2, description=…

The above does not fully solve our problem, which is finding an effective invariant for stablecoins. It does not solve it effectively for a number of reasons. In constant product, for example, in the case of two coins, we have asymptotes along y and along x axes, and this ensures we always have inventory. When terms with $\chi$ start to quickly overpower the constant-product, we get a pure constant-sum invariant.

To remedy, let's make leverage dynamic. It would make sense that we have constant-sum as much as possible when the price is equal to 1, and even if move farther, we still should get low slippage, but as soon as we deplete our inventory, we must move to the constant product invariant, instead. Let's introduce the following dynamic definition of $\chi$

$$\chi = \frac{A \prod x_i}{D / n^n} \tag{2}$$

Subbing this into our equation from before, we have

$$A n^n \sum x_i + D = ADn^n + \frac{D^{n + 1}}{n^n \prod x_i}$$